In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import rasterio
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

MAPPING_FILE = Path("image_feature_map.json")
STATS_CSV = Path("mask/segmentation_stats.csv")
MASKS_DIR = Path("mask/masks")
OVERLAYS_DIR = Path("mask/overlays")
IM_DIR = Path("converted_sat_images")

In [2]:
# Load mapping + stats
mapping = {}
if MAPPING_FILE.exists():
    mapping = json.loads(MAPPING_FILE.read_text())

stats_df = pd.DataFrame()
if STATS_CSV.exists():
    stats_df = pd.read_csv(STATS_CSV)

# prepare images list
images = sorted(list({str(p) for p in mapping.keys()} | set(stats_df["image"].unique().tolist()))
, key=lambda s: str(s))
images = [i for i in images if i and Path(i).exists()]

KeyError: 'image'

In [ ]:
def load_image_pil(img_path, max_size=(1024,1024)):
    img = Image.open(img_path).convert("RGBA")
    img.thumbnail(max_size, Image.LANCZOS)
    return img

def load_mask_arr(mask_path):
    if not Path(mask_path).exists():
        return None
    try:
        with rasterio.open(str(mask_path)) as src:
            arr = src.read(1)
            return arr
    except Exception:
        return None

def color_for_feature(name, seed=None):
    seed_val = abs(hash(name)) % (2**32) if seed is None else seed
    rng = np.random.RandomState(seed_val)
    return tuple(int(x) for x in rng.randint(50, 230, size=3))

def overlay_masks_on_image(img_pil, masks_dict, alpha=0.5):
    # masks_dict: {feature_name: mask_arr}
    W, H = img_pil.size
    base = img_pil.copy().convert("RGBA")
    composite = Image.new("RGBA", (W, H), (0,0,0,0))

    for feat, arr in masks_dict.items():
        if arr is None:
            continue
        # Ensure arr same size; if not, resize to W,H
        h, w = arr.shape
        if (w, h) != (W, H):
            # resample with rasterio would be ideal; use PIL nearest for speed
            arr_img = Image.fromarray(arr.astype(np.uint8)).resize((W, H), Image.NEAREST)
            arr = np.array(arr_img)

        mask_bool = arr > 0
        if mask_bool.sum() == 0:
            continue
        color = color_for_feature(feat)
        rgba = np.zeros((H, W, 4), dtype=np.uint8)
        rgba[..., 0][mask_bool] = color[0]
        rgba[..., 1][mask_bool] = color[1]
        rgba[..., 2][mask_bool] = color[2]
        rgba[..., 3][mask_bool] = int(255 * alpha)
        mask_img = Image.fromarray(rgba, mode="RGBA")
        composite = Image.alpha_composite(composite, mask_img)

    # Composite masks onto original
    out = Image.alpha_composite(base, composite)
    return out


In [ ]:
def show_image_and_overlays(img_path, selected_features=None):
    clear_output(wait=True)
    print(f"Image: {img_path}")
    orig = load_image_pil(img_path)
    # Get features to visualize — prefer mapping, fallback to stats rows for this image
    if selected_features is None:
        selected_features = mapping.get(img_path, [])
    # Build masks dict
    masks = {}
    missing = []
    for feat in selected_features:
        safe = feat.replace(" ", "_")
        mask_path = MASKS_DIR / f"{Path(img_path).stem}__{safe}_masks.tif"
        # If file not found, try to read stats csv reference path
        if not mask_path.exists() and not stats_df.empty:
            row = stats_df[(stats_df.image == img_path) & (stats_df.feature == feat)]
            if not row.empty:
                candidate = Path(str(row.iloc[0].mask_file))
                if candidate.exists():
                    mask_path = candidate
        arr = load_mask_arr(mask_path) if mask_path.exists() else None
        masks[feat] = arr
        if arr is None:
            missing.append(feat)

    # Compose overlays only for features with masks
    overlay = overlay_masks_on_image(orig, {k:v for k,v in masks.items() if v is not None})
    display(overlay)
    # Show table of stats for selected features
    if not stats_df.empty:
        display(stats_df[stats_df.image == img_path][["feature", "n_objects", "mask_pixels", "coverage_pct", "mean_score"]])

    if missing:
        print("No masks for:", missing)

In [ ]:
img_dropdown = widgets.Dropdown(options=images, description="Image", layout=widgets.Layout(width="90%"))
# features list will update when image selection changes
checkbox_container = widgets.VBox([])

def build_feature_checkboxes(img_path):
    img_features = mapping.get(img_path, [])
    # fallback to stats file for features detected
    if not img_features and not stats_df.empty:
        img_features = stats_df[stats_df.image == img_path]["feature"].tolist()
    cbs = [widgets.Checkbox(value=True, description=f) for f in img_features]
    return cbs

def on_image_change(change):
    if change['name'] != 'value':
        return
    img_path = change['new']
    # Build checkboxes
    cbs = build_feature_checkboxes(img_path)
    if not cbs:
        checkbox_container.children = [widgets.Label(value="No features mapped for this image")]
    else:
        checkbox_container.children = cbs
    # Auto display with all enabled
    selected = [cb.description for cb in cbs if cb.value]
    show_image_and_overlays(img_path, selected_features=selected)

def on_toggle_checked(change):
    # recompute overlay for currently selected image
    img_path = img_dropdown.value
    selected = [cb.description for cb in checkbox_container.children if isinstance(cb, widgets.Checkbox) and cb.value]
    show_image_and_overlays(img_path, selected_features=selected)

# Connect observers
img_dropdown.observe(on_image_change, names='value')

# When checkboxes change, re-render
def attach_checkbox_observers():
    for cb in checkbox_container.children:
        if isinstance(cb, widgets.Checkbox):
            cb.observe(on_toggle_checked, names='value')

# Compose and show UI
display(img_dropdown)
display(checkbox_container)

# Initialize
if images:
    img_dropdown.value = images[0]
    # attach observers to checkboxes after creation
    attach_checkbox_observers()